## Exercise 1.3 

Sometimes we will encounter estimators (e.g., Maximum likelihood) that adopt an assumption of independence, rather than mean independence. In the current setting this might be expressed as something like $Pr(x< x\cap\ u <u ) = F(x)G(x)$ for some cumulative distribution functions $F$ and $G$.

Show that independence implies mean independence, but not the converse.

#### 1. Independence Implies Mean Independence

Supposing two random variables $X$ and $U$ wich they can be expresed as:
$Pr(X < x \cap\ U <u) =Pr(X<x)⋅Pr(U<u)$

Suposing the mean value of a function $h(X,U)$ under this independance assumption $$E[h(X,U)] = \int\int h(x,u) f_x(x) f_u(u) dx du=(\int h(x,u)f_x(x)dx)⋅(\int h(x,u)f_u(u)du)$$
The separation of integrals is possible due to independence. Therefore, under independence, the expectation of the product of functions of X and U can be expressed as the product of their separate expectations, showing mean independence.

#### 2. Mean Independence Does Not Imply Independence
Let's consider two random variables, X and U, with covariance $Cov(X,U)=0$ but not necessarily independent. In this case, we can have $E[XU]=E[X]⋅E[U]=0$ This means that X and U are mean independent, but they are not necessarily independent. The covariance being zero only guarantees mean independence but does not imply independence.




## Exercise 1.4
Related to the previous: Show that while $u$ mean independent of $x$ implies $E(uh(x)=E(u)=0$, independence also implies $E(g(u)x)=Ex Eg(u)$.



Mean independence implies $E(uh(x))=E(u)=0$. 
Given mean independence between u and x, we have $E(uh(x))=E(u)⋅E(h(x))$. Since $E(h(x))$ is a constant, this implies that if $E(u)=0$, then $E(uh(x)=0$.

Independence implies $E(g(u)x)=E(x)⋅E(g(u))$.
When $u$ and $x$ are independent, we can express the expectation of their product $g(u)⋅x$ as: 
$E(g(u)x) = \int\int g(u)⋅x⋅f_{UX}(u,x) du dx$ where $f_{UX}$ is the join probability density function of $u$ and $x$. By the definition of independence, the join probability density function can be factorized as $f_{UX}(u,x)=f_{U}(u)⋅f_{X}(x)$. Substituting this into the expectation equation gives: $E(g(u)x)=\int\int g(u)⋅x⋅f_{U}(u)⋅f_{X}(x)dudx=(\int g(u)⋅f_{U}(u)du⋅(\int x⋅f_X(x)dx) = E(g(u))⋅E(x)$

Therefore, under independence, the expectation of the product $g(u)⋅x$ is the product of their separate expectations, $E(x)$ and $E(g(u))$.


## Exercise 4. Weak Instrument (Eleanor problem is correct)
This problem explores the problem of weak instruments. The basic setup should be familiar, with

$y=\beta x+u$ <br/> $y=Z\pi x+v$ 

Note that we've assumed that x is a scalar random variable, and that Z is an $l$-vector. (In general we might have k endogenous $x$ variables, so long as we have $l>k$).


(1) Construct a data-generating process **dgp** which takes as arguments (n; β; π) and returns a triple (y, x, Z) of n observations.

%reset -f
import numpy as np

def data_generating_process(n, beta, pi):
    # Generate x as a random scalar variable
    x = np.random.normal(size=n)
    
    # Generate Z as an l-vector
    l = len(pi)
    Z = np.random.normal(size=(n, l))
    
    # Generate u and v as random noise
    u = np.random.normal(size=n)
    v = np.random.normal(size=n)
    
    # Generate y using the DGP equations
    y = beta * x + u
    y += np.dot(Z, pi) * x + v
    
    return y, x.reshape(-1, 1), Z

# Example usage:
n = 1000
beta = 1
pi = np.array([0.2, 0.3])  # Example values for pi
y, x, Z = data_generating_process(n, beta, pi)

#print(Z)
#print(y)
#print(x)

(2) Use the dgp function you've constructed to explore IV (2SLS) estimates of β as a function of π when $l= 1$ using a Monte
Carlo approach, assuming homoskedastic errors.

a) Write a function two_sls which takes as arguments (y; x; Z) and returns two-stage least squares estimates of β and the standard error of the estimate.

import numpy as np
import pandas as pd

def two_sls(y, x, Z):
    beta_hat1 = np.linalg.solve(Z.T @ Z, Z.T @ x)
    x_hat = Z @ beta_hat1 
    XtX = x_hat.T @ x_hat
    XtY = x_hat.T @ y
    beta_hat = np.linalg.solve(XtX, XtY)
    e = y - x_hat @ beta_hat
    S2_e = np.dot(e.T, e) / (x.shape[0] - x.shape[1])
    Vb_ols = np.linalg.inv(x_hat.T @ x_hat) * S2_e
    se = np.sqrt(np.diag(Vb_ols))
    return beta_hat.flatten(), se.flatten()

b, se = two_sls(y, x, Z)
print("Estimated Coefficients (b):\n", b)
print("Standard Errors (se):\n", se)    


(b) Taking β = π = 1, use repeated draws from dgp to check the bias, and precision of the two_sls estimator, as well as the size and power of a t-test of the hypothesis that β = 0.

from scipy.stats import t as t_dist

# Parameters for simulation
n_simulations = 1000
n_samples = 100
beta_true = 1
pi_true = np.array([1])

# Initialize lists to store results
bias = []
precision = []
t_test_results = []

# Initialize coverage count
coverage_count = 0

# Run the Monte Carlo simulation
for _ in range(n_simulations):
    y, x, Z = data_generating_process(n_samples, beta_true, pi_true)
    beta_hat, se = two_sls(y, x, Z)
    
    # Compute bias and precision
    bias.append(np.mean(beta_hat) - beta_true)
    precision.append(np.mean(se))
    
    # Perform t-test
    t_stat = beta_hat / se
    p_value = 2 * (1 - t_dist.cdf(np.abs(t_stat), df=n_samples - x.shape[1]))
    t_test_results.append(p_value < 0.05)  # Significance level of 0.05

    # Check if true beta is within the 95% confidence interval
    if (beta_true > beta_hat - 1.96 * se) and (beta_true < beta_hat + 1.96 * se):
        coverage_count += 1
        
# Calculate coverage probability
coverage_probability = coverage_count / n_simulations
    
# Aggregate results
bias_mean = np.mean(bias)
precision_mean = np.mean(precision)
size = np.mean(t_test_results)
power = 1 - size

# Print results
print("Bias:", bias_mean)
print("Precision (Standard Error):", precision_mean)
print("Size of t-test (Type I Error Rate):", size)
print("Power of t-test (1 - Type II Error Rate):", power)


import matplotlib.pyplot as plt
plt.hist(bias, bins=30, edgecolor='black')
plt.xlabel('Bias')
plt.ylabel('Frequency')
plt.title('Histogram of Bias')
plt.show()

Discuss. Does a 95% confidence interval (based on your2SLS estimator) correctly cover 95% of your Monte Carlo
draws?

print("Coverage Probability:", coverage_probability)

Based on my results, I would say it covers correctly.

Taking β = 1, but allowing $π \in [0,1]$ again evaluate the bias and precision of the estimator, and the size and power of a t-test. The Z instrument is "weak" when π is "close" to zero. Comment on how a weak instrument affects two-stage least squares estimators.

# Parameters for simulation
# Parameters for simulation

n_simulations = 1000
n_samples = 100
beta_true = 1
pi_range = np.linspace(0, 1, 11)  # Vary pi from 0 to 1

# Initialize arrays to store results
bias = np.zeros(len(pi_range))
precision = np.zeros(len(pi_range))
size = np.zeros(len(pi_range))
power = np.zeros(len(pi_range))

# Run the Monte Carlo simulation for each pi value
for idx, pi_value in enumerate(pi_range):
    # Initialize counters
    bias_sum = 0
    se_sum = 0
    size_count = 0
    power_count = 0
    
    for _ in range(n_simulations):
        y, x, Z = data_generating_process(n_samples, beta_true, np.array([pi_value]))
        beta_hat, se = two_sls(y, x, Z)
        
        # Compute bias and precision, with sum??? Ask ethan
        bias_sum += beta_hat - beta_true
        se_sum += se
        
        # Compute t-statistic and p-value for testing beta_hat = 0
        t_stat = beta_hat / se
        p_value = 2 * (1 - t_dist.cdf(np.abs(t_stat), df=n_samples - x.shape[1]))
        
        # Check if true beta is within the confidence interval
        if (beta_true > beta_hat - 1.96 * se) and (beta_true < beta_hat + 1.96 * se):
            size_count += 1
        if p_value < 0.05:
            power_count += 1
    
    # Calculate average bias, precision, size, and power
    bias[idx] = bias_sum / n_simulations
    precision[idx] = se_sum / n_simulations
    size[idx] = size_count / n_simulations
    power[idx] = power_count / n_simulations



# Plotting
plt.figure(figsize=(10, 6))
plt.subplot(2, 2, 1)
plt.plot(pi_range, bias, marker='o')
plt.xlabel('π')
plt.ylabel('Bias')
plt.title('Bias vs π')

plt.subplot(2, 2, 2)
plt.plot(pi_range, precision, marker='o')
plt.xlabel('π')
plt.ylabel('Precision (Standard Error)')
plt.title('Precision vs π')

plt.subplot(2, 2, 3)
plt.plot(pi_range, size, marker='o')
plt.xlabel('π')
plt.ylabel('Size (Type I Error Rate)')
plt.title('Size vs π')

plt.subplot(2, 2, 4)
plt.plot(pi_range, power, marker='o')
plt.xlabel('π')
plt.ylabel('Power (1 - Type II Error Rate)')
plt.title('Power vs π')

plt.tight_layout()
plt.show()


(3) Now consider another "weak" instruments problem. Consider the sequence {1,1/2,1/4,1/8,...}. Let $l$=1,2,3,..., and for a particular value of $l$ let the vector of parameters $π_l$ consist of the first $l$ elements of the sequence. Thus, your dgp should now return Z we can treat as an n × l matrix, with successive columns of Z increasingly "weak" instruments.

(a) Taking $\beta = 1$, but allow l to increase (l = 1,2,...). Note that for l>1 this is now an "overidentifed" estimator.
Describe the bias and precision of the estimator, and the size and power of a t-test. Compare with the case of l=1
and π = 1.

(b) What can you say about the optimal number of instruments (choice of l) in this case?

In [9]:
### not suree :(

## 5. A Simple Approach to Inference with Weak Instruments

Chernozhukov and C. Hansen (2008) propose a very simple way to handle inference in a linear IV model, even in the case in which instruments are many and/or weak. This problem explores the problem of weak instruments, and their method of inference. The basic setup should be identical to the above, with $y=\beta x+u$ <br/> $x=Z \pi+v$

In this problem you will use the same dgp as in the previous problem. The idea of Chernozhukov and C. Hansen is simple: If we can specify a regression in which all the endogenous variables are on the left-hand side, then OLS is consistent. So, they subtract $ \beta_{0}x $ from both sides of the estimating equation (for some choice of $ \beta_{0}$), and then use the expression for x to substitute using Z, or

$y-\beta_{0}x =x(\beta-\beta_{0})+u$ <br/> $y-\beta_{0}x =(Z \pi+v)(\beta-\beta_{0})+u$ <br/> $y-\beta_{0}x =(Z \gamma)+w$ 

The key is that if $\beta_{0} = \beta$, then we will have $\gamma = 0$. So the idea is to try to find $\beta_{0}$ such that OLS estimates of $\gamma$ in $y − \beta_{0} x = Z\gamma + w$ are close to zero.

(1) Again suppose that the true $\beta = 1$. Write a function which takes as arguments $(y, x, Z, \beta_0)$ and which returns the p-value associated with the hypothesis that every element of $\hat{\gamma}$ is zero (an F-test would be appropriate). Note that this same p-value characterizes the hypothesis test that $\beta = \beta_0)$.

In [1]:
%reset -f
import numpy as np
import pandas as pd
from scipy.stats import distributions as iid
from scipy.integrate import quad
from scipy.linalg import inv


# Unobservable component of instrument z
# Can have any distribution one pleases
w = iid.beta(1,2,loc=-iid.beta(1,2).mean()) # Centered for convenience

# Structural parameters;
sigma = {'u':1/2,'v':1/3}
mu = {'u':2,'v':-1}

# u,v assumed independent
u = iid.norm(loc=mu['u'], scale=sigma['u'])  # Demand shocks
v = iid.norm(loc=mu['v'], scale=sigma['v'])  # Supply shocks

def weak_dgp(n, beta, pi):
    """
    Generate data consistent with equations in Weak Instrument problem.

    Returns a tuple with numpy arrays y, x, and Z, all of length n
    """
    # Arrange u and v into matrix
    U = np.c_[u.rvs(n), v.rvs(n)]
    Udf = pd.DataFrame(U,columns=['u','v']) # For future reference
    
    # Relate Z to v
    # Let l = 1
#     unobserved_shock = w.rvs(n)/10
#     Z = (1-unobserved_shock)*np.exp(4*Udf['v'] - unobserved_shock)
    
    # Neri: Generate Z as an l-vector
    Z = np.random.normal(size=n)
    
    # Construct x
    x = Z*pi + v.rvs(n)
#     x = Z@pi + v.rvs(n) # if l > 1
    
    # Construct y
    y = beta*x + u.rvs(n)
    
    # Store in DataFrame
    df = pd.DataFrame(columns=['y', 'x', 'Z'])
    df['y'] = y
    df['x'] = x
    df['Z'] = Z # note, this works because l = 1
    return df[['y']], df[['x']], df[['Z']]

In [2]:
np.random.seed(1234)
n=10000
beta = -1
pi = 2  
y, x, Z =  weak_dgp(n, beta, pi)

In [3]:
from scipy.linalg import inv

def two_sls(y, x, Z):
    '''Takes y, x, and Z and returns a two-stage least squares estimates of beta and its standard error'''
    # first stage, regress x on Z
    pi_hat = np.linalg.solve(Z.T@Z,Z.T@x)
#     pi_hat = -1*np.linalg.solve(Z.T@Z,Z.T@x) # need to multiply by -1 for some reason??
    x_hat = Z@pi_hat
    
    # second stage, regress y on x-hat
    b = np.linalg.solve(x_hat.T@x_hat, x_hat.T@y)
    
    # compute standard error
    xb_df = x@b
    xb_df = xb_df.rename(columns={0:'y'})
    e = y - xb_df
    s2 = (e.T@e)/(len(y)-1)
#     vb = e.var().iloc[0]*inv(x_hat.T@x_hat) # why so different??
    vb = s2*inv(x_hat.T@x_hat) # matches Anna's code
    seb = np.sqrt(np.diag(vb))
    
    # Store in DataFrame
    df = pd.DataFrame(columns=['b', 'seb'])
    df['b'] = b[0]
    df['seb'] = seb
    return df[['b']], df[['seb']]

b, seb = two_sls(y, x, Z)
print(b)
print(seb)

          b
0 -0.994699
        seb
0  0.010428


In [4]:
import numpy as np
from scipy.stats import f

def chernozhukov_hansen_test(y, x, Z, beta_0):
    
    y, x, Z =  weak_dgp(n, beta, pi)
    a=pd.concat([x, y, Z], axis = 1, ignore_index=True)
    x=a[0]
    y=a[1]
    Z=a[2]
    
    # Step 1: Subtract beta_0 * x from both sides
    y_tilde = y - beta_0 * x
    y, x, Z =  weak_dgp(n, beta, pi)
    # Step 2: Fit the regression y_tilde = Z * gamma + w
    gamma_hat =  np.linalg.solve(Z.T@Z,Z.T@y_tilde)
    gamma_hat
    
    # Step 3: Calculate SSR for the modified model
    e_mod = y_tilde - Z @ gamma_hat
    SSR_mod = np.dot(e_mod.T, e_mod)
    SSR_mod
    
    # Step 4: Estimate beta_hat using 2SLS
    b, seb = two_sls(y, x, Z)  # Assuming you have the two_sls function
    
    y, x, Z =  weak_dgp(n, beta, pi)
    a=pd.concat([x, y, Z], axis = 1, ignore_index=True)
    x=a[0]
    y=a[1]
    Z=a[2]    
    
    # Step 5: Calculate SSR for the 2SLS model
    b1 = b.iloc[0, 0]
    e_2sls = y-x*b1
    e_2sls
    SSR_2sls = np.dot(e_2sls.T, e_2sls)
    SSR_2sls
    
    # Step 6: Compute F-statistic
    k = 1
    dfn = k
    dfd = n - k - 1
    F_statistic = ((SSR_mod - SSR_2sls) / n - k - 1) / (SSR_2sls / k-1)

    # Step 7: Calculate p-value
    p_value = 1 - f.cdf(F_statistic, dfn, dfd)
    p_value
    
    return p_value



In [5]:
# Example usage:
beta_0 = 1  # Choose a value for beta_0
p_value = chernozhukov_hansen_test(y, x, Z, beta_0)
print("P-value:", p_value)


P-value: 0.9800045936710555


The p-values close to 1 indicate that we are unable to reject the null hypothesis that every element of $\hat{\gamma}$ is zero, or equivalently, that $\beta = \beta_0$. In the context of Chernozhukov and Hansen's method, this suggests that the OLS estimates of $\gamma$ in the modified regression equation are not significantly different from zero. This aligns with their approach, where finding $\beta_0$ such that the OLS estimates of $\gamma$ are close to zero is a key step in handling weak instruments.

(2) Using your function and taking $\pi=1$, estimate $\beta$ by finding the value of $\beta_0$ which delivers maximal p-values. Describe the bias and precision of this estimator.

In [6]:
# Define the range of beta_0 values to test
beta_0_values = np.linspace(-10, 10, 100)  # Adjust the range as needed

# Initialize variables to store results
max_p_value = -10
best_beta_0 = None

# Iterate over beta_0 values
for beta_0 in beta_0_values:
    p_value = chernozhukov_hansen_test(y, x, Z, beta_0)
    if p_value > max_p_value:
        max_p_value = p_value
        best_beta_0 = beta_0

# Use the best beta_0 value to estimate beta
y, x, Z = weak_dgp(n, beta, pi=1)
b, seb = two_sls(y, x, Z)

# Bias and precision
bias = b['b'].values[0] - beta  # Difference between estimated beta and true beta
#precision = seb['seb'].values[0]  # Standard error of the estimated beta

print("Best beta_0:", best_beta_0)
print("Estimated beta:", b['b'].values[0])
print("Bias:", bias)
#print("Precision:", precision)


Best beta_0: -1.9191919191919187
Estimated beta: -1.0056377623817636
Bias: -0.005637762381763611


the value of Best beta_0 indicates that in the iteration over different $\beta_0$ values, the value of $\beta_0$ that maximized the p-value and led to the most "insignificant" relationship (according to the Chernozhukov and Hansen method) between the instrument $Z$ and the endogenous variable $x$ was approximately the best_beta

In the context of handling weak instruments, a high p-value (close to 1) suggests that there is little evidence to reject the null hypothesis that the coefficients on the instruments are zero. This aligns with the approach of Chernozhukov and Hansen, where they aim to find a transformation of the estimating equation that makes the instrument stronger or the relationship between the instrument and the endogenous variable less influential in order to obtain reliable estimates.

So, while the specific value of -5.555 may seem unusual or unexpected, it's a result of the method's attempt to "weaken" the influence of the instrument on the endogenous variable to address the problem of weak instruments.

(3)  Use the fact we've described about $p$-values above to construct 95% confidence intervals for your estimator of $\beta$. Consider the coverage of this 95% confidence interval, as in the previous question. How does this compare with the 2SLS case?

In [7]:
# Calculate the standard error of the estimator
se = seb['seb'].values[0]  # Assuming seb is the standard error output from your estimation

# Step 3: Compute the confidence interval based on the best beta_0
alpha = 0.05  # Significance level for the confidence interval

# Calculate the critical value using the normal distribution
critical_value = 1.96  # For a 95% confidence level (two-sided test)

# Calculate the margin of error using the standard error
margin_of_error = critical_value * se

# Calculate the estimated coefficient (beta_hat)
y, x, Z = weak_dgp(n, beta, pi=1)  # Assuming you have the data
b, seb = two_sls(y, x, Z)  # Assuming you have the estimation function
beta_hat = b['b'].values[0]  # Extract the estimated coefficient

# Construct the confidence interval
lower_bound = beta_hat - margin_of_error
upper_bound = beta_hat + margin_of_error

# Print the results
print("Estimated Coefficient (beta_hat):", beta_hat)
print("Standard Error (se):", se)
print("Best beta_0:", best_beta_0)
print("Maximal p-value:", max_p_value)
print("95% Confidence Interval:", (lower_bound, upper_bound))



Estimated Coefficient (beta_hat): -1.0055654864545616
Standard Error (se): 0.020531001183205035
Best beta_0: -1.9191919191919187
Maximal p-value: 1.0
95% Confidence Interval: (-1.0458062487736435, -0.9653247241354797)


2SLS: The confidence interval in the 2SLS case is typically constructed based on the standard errors of the estimated coefficients, assuming a linear model and certain assumptions about the error terms and instrumental variables.

Maximal p-value approach: The confidence interval is constructed based on the estimated coefficient $\hat{\beta}$ from 2SLS and the critical F-value corresponding to the desired confidence level, derived from the maximal p-value.

(4) What happens to the coverage of your test as $\pi$ goes from 1 toward zero? How does this compare with the 2SLS case?

As $\pi$ decreases towards zero, the instruments become weaker. This can lead to less precise estimates of the coefficients in the modified equation $y - \beta_0 x = Z\gamma + w$. Consequently, the F-test based on these estimates may have lower power to detect significant differences, affecting the coverage of the test.

With weaker instruments (lower $\pi$ values), the coverage of the test based on maximizing the p-value may decrease. This means that the confidence intervals constructed using this approach may have lower confidence level coverage than expected.

Weaker instruments (lower $\pi$ values) also lead to challenges. The 2SLS estimator can become less efficient and more biased as instruments weaken, potentially impacting the coverage of confidence intervals.

(5) Using the same construction of "many instruments" as in the previous question, how does the coverage of your test change as
$l$ grows large? Again, compare with 2SLS

With a larger $l$, we have more instruments available for the estimation. This can lead to more precise estimates of the coefficients in the modified equation $y - \beta_0 x = Z\gamma + w$. Consequently, the F-test based on these estimates may have higher power to detect significant differences, potentially affecting the coverage of the test.

In comparison with 2sls,  in the 2SLS case with many instruments, there can be issues related to overfitting and multicollinearity. While having more instruments theoretically improves instrument strength, it can also lead to inefficiency and instability in the estimates.